In [9]:
data_dir = '/home/vinay/Downloads/projects/data/'
import pandas as pd
df_wz = pd.read_csv(data_dir+'orderbook_resolution_5_2021-03-01_2021-03-24_wazirx.csv')
df_wz['timestamp'] = pd.to_datetime(df_wz['timestamp'])

In [10]:
df_wz.head(4)

,timestamp,exchange,symbol,bid_0_price,ask_0_price,midprice
0,2021-03-01 20:32:05,wazirx,USDTINR,76.76,76.9,76.83
1,2021-03-01 20:32:05,wazirx,BTCINR,3710743.00,3716553.0,3713648.00
2,2021-03-01 20:32:05,wazirx,ETHINR,117505.00,117700.0,117602.50
3,2021-03-01 20:32:10,wazirx,BTCINR,3710743.00,3716553.0,3713648.00


In [11]:
df_ftx = pd.read_csv(data_dir+'orderbook_resolution_5_2021-03-01_2021-03-24_ftx.csv')
df_ftx['timestamp'] = pd.to_datetime(df_ftx['timestamp'])
df_ftx.head(4)

,timestamp,exchange,symbol,bid_0_price,ask_0_price,midprice
0,2021-03-01 20:32:05,ftx,ETHUSD,1534.8000,1535.2000,1535.00000
1,2021-03-01 20:32:05,ftx,BTCUSD,48344.0000,48351.0000,48347.50000
2,2021-03-01 20:32:05,ftx,USDTUSD,1.0011,1.0012,1.00115
3,2021-03-01 20:32:10,ftx,ETHUSD,1534.9000,1535.0000,1534.95000


In [7]:
len(df_wz),len(df_ftx)

(1062827, 1151397)

In [50]:
wzr_btc = df_wz[df_wz['symbol'] == 'BTCINR'][['timestamp','midprice']].copy().rename({'midprice':'midbtc'},axis=1).set_index('timestamp')
wzr_eth = df_wz[df_wz['symbol'] == 'ETHINR'][['timestamp','midprice']].copy().rename({'midprice':'mideth'},axis=1).set_index('timestamp')
usd_inr = df_wz[df_wz['symbol'] == 'USDTINR'][['timestamp','midprice']].copy().rename({'midprice':'usdinr'},axis=1).set_index('timestamp')
ftx_btcusd = df_ftx[df_ftx['symbol'] == 'BTCUSD'][['timestamp','midprice']].copy().rename({'midprice':'ftx_midbtc_usd'},axis=1).set_index('timestamp')
ftx_ethusd = df_ftx[df_ftx['symbol'] == 'ETHUSD'][['timestamp','midprice']].copy().rename({'midprice':'ftx_mideth_usd'},axis=1).set_index('timestamp')


In [118]:
t = pd.concat([wzr_btc,wzr_eth,usd_inr,ftx_btcusd,ftx_ethusd],join='inner',axis=1)
t['midbtc_ftx'] = t['ftx_midbtc_usd']*t['usdinr']
t['mideth_ftx'] = t['ftx_mideth_usd']*t['usdinr']
t.drop(labels=['ftx_midbtc_usd','ftx_mideth_usd'],axis=1,inplace=True)

In [119]:
t['time'] = pd.to_datetime(t.index)
t.head(2)

,midbtc,mideth,usdinr,midbtc_ftx,mideth_ftx,time
timestamp,,,,,,
2021-03-01 20:32:05,3713648.0,117602.5,76.830,3714538.425,117934.05000,2021-03-01 20:32:05
2021-03-01 20:32:10,3713648.0,117602.5,76.745,3711771.925,117799.73775,2021-03-01 20:32:10


In [139]:
# Checking to see if all the data is at 5s resolution.
time_diff = t['time'].diff(1).dt.total_seconds().dropna()
bad_ids = (time_diff != 5.0)
time_diff[bad_ids]

timestamp
2021-03-01 20:34:35    65.0
2021-03-01 20:45:55    10.0
2021-03-01 21:01:55    10.0
2021-03-01 21:35:15    10.0
2021-03-01 21:35:30    15.0
                       ... 
2021-03-24 12:36:00    10.0
2021-03-24 12:36:55    15.0
2021-03-24 12:37:25    15.0
2021-03-24 12:42:25    10.0
2021-03-24 12:57:50    60.0
Name: time, Length: 4266, dtype: float64

In [140]:
time_diff[bad_ids].describe()

count     4266.000000
mean       109.968354
std       1767.093090
min         10.000000
25%         10.000000
50%         50.000000
75%         65.000000
max      78650.000000
Name: time, dtype: float64

In [134]:
inds = []
for ind,ele in enumerate(time_diff==78650.000000):
    if ele == True:
        inds.append(ind)
        

In [137]:
t.iloc[inds[0]-2:inds[0]+3]

,midbtc,mideth,usdinr,midbtc_ftx,mideth_ftx,time
timestamp,,,,,,
2021-03-09 19:45:00,4039499.5,135695.0,74.125,4.047633e+06,135963.78125,2021-03-09 19:45:00
2021-03-09 19:45:05,4039499.5,135695.0,74.125,4.047522e+06,135937.83750,2021-03-09 19:45:05
2021-03-09 19:45:10,4039499.5,135695.0,74.125,4.046373e+06,135882.24375,2021-03-09 19:45:10
2021-03-10 17:36:00,4077551.5,134850.0,74.130,4.092310e+06,135387.32550,2021-03-10 17:36:00
2021-03-10 17:36:05,4077551.5,134850.0,74.130,4.092384e+06,135398.44500,2021-03-10 17:36:05


In [141]:
np.unique(time_diff[bad_ids])

array([1.0000e+01, 1.5000e+01, 2.0000e+01, 2.5000e+01, 3.0000e+01,
       3.5000e+01, 4.0000e+01, 4.5000e+01, 5.0000e+01, 5.5000e+01,
       6.0000e+01, 6.5000e+01, 7.0000e+01, 7.5000e+01, 8.0000e+01,
       8.5000e+01, 9.0000e+01, 9.5000e+01, 1.0000e+02, 1.0500e+02,
       1.1000e+02, 1.1500e+02, 1.2000e+02, 1.2500e+02, 1.3000e+02,
       1.3500e+02, 1.4000e+02, 1.4500e+02, 1.5000e+02, 1.5500e+02,
       1.6000e+02, 1.6500e+02, 1.7000e+02, 1.7500e+02, 1.8000e+02,
       1.8500e+02, 1.9000e+02, 1.9500e+02, 2.0000e+02, 2.0500e+02,
       2.1000e+02, 2.1500e+02, 2.2000e+02, 2.2500e+02, 2.3500e+02,
       2.4000e+02, 2.5000e+02, 2.6000e+02, 2.7500e+02, 2.8000e+02,
       2.8500e+02, 2.9000e+02, 3.0000e+02, 3.1000e+02, 3.1500e+02,
       3.3500e+02, 3.4000e+02, 3.5000e+02, 3.6500e+02, 5.7500e+02,
       7.3000e+02, 1.6200e+03, 3.4750e+03, 3.6050e+03, 5.2650e+03,
       9.3300e+03, 1.2320e+04, 1.2780e+04, 1.5225e+04, 2.4795e+04,
       3.2955e+04, 6.9110e+04, 7.8650e+04])

In [148]:
t['date'] = t['time'].dt.date
t.groupby('date').count()['time']


date
2021-03-01     2195
2021-03-02    15314
2021-03-03    12718
2021-03-04    16045
2021-03-05    15843
2021-03-06    15999
2021-03-07    15917
2021-03-08    13708
2021-03-09    12101
2021-03-10     4457
2021-03-11     9127
2021-03-12     5115
2021-03-13    15690
2021-03-14    15112
2021-03-15    14570
2021-03-16    10949
2021-03-17    14559
2021-03-18    10845
2021-03-19    13681
2021-03-20    14775
2021-03-21    14418
2021-03-22    14902
2021-03-23    15382
2021-03-24     9019
Name: time, dtype: int64

## This data is fucked up so badly..welcome to real world.

### let's just see the plane wazirx data


In [157]:
df_wz['date'] = df_wz['timestamp'].dt.date
df_wz[df_wz['symbol'] == 'BTCINR'].groupby('date').count()['timestamp']

date
2021-03-01     2350
2021-03-02    16292
2021-03-03    13300
2021-03-04    16746
2021-03-05    16686
2021-03-06    16607
2021-03-07    16643
2021-03-08    16324
2021-03-09    13048
2021-03-10     4531
2021-03-11    16586
2021-03-12    14850
2021-03-13    16545
2021-03-14    16404
2021-03-15    16108
2021-03-16    11842
2021-03-17    16524
2021-03-18    16832
2021-03-19    15032
2021-03-20    16218
2021-03-21    15992
2021-03-22    16359
2021-03-23    16493
2021-03-24     9231
Name: timestamp, dtype: int64

### Even orginal data is reasonably fucked.